In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

%config Completer.use_jedi = False 


#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


# Settings for the multiple linear regression model (MLR)
#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    alpha, date_exp_str = arguments.split(',')
    
    alpha              = float(alpha)   
    
elif run == 'manual':
    # Enter arguments manually 
    alpha              = 0.01
    date_exp_str       = '230207_Full'
    remove_data        = True

# Mortality group?
if ('Full' in date_exp_str):
        
    print(date_exp_str, alpha)    

230104_Full 0.01


In [3]:
import glob

file_list = glob.glob('spe_reg_results/'+date_exp_str+'_*'+str(alpha)+'*_SPE_best_Models_dAIC.pkl')
file_list = [f for f in file_list if 'False' not in f]
file_list

['spe_reg_results/230104_Full_NoInsectsFire_0.01_True_SPE_best_Models_dAIC.pkl',
 'spe_reg_results/230104_Full_FireOnly_0.01_True_SPE_best_Models_dAIC.pkl',
 'spe_reg_results/230104_Full_0.01_True_SPE_best_Models_dAIC.pkl',
 'spe_reg_results/230104_Full_NoFire_0.01_True_SPE_best_Models_dAIC.pkl']

In [4]:
#---------------#
#  Import data  #
#---------------#
mtf_type = ['count','cmass']
mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']


# Initialise storage
res_tables  = np.empty((len(mortality_groups),len(mtf_type)),dtype='object') # table; res_objects
df_res_mort = pd.DataFrame()
for f,file in enumerate(file_list):
    
    with open(file,'rb') as l:
        res_temp = pickle.load(l)
        
        for t in range(2):
            res_tables[f,t] = res_temp[t]

            
        res_tables_formatted = res_tables.copy()
        df_mort = pd.DataFrame()
        for t,tpe in enumerate(mtf_type):
            res_tables_formatted[f,t] = res_tables[f,t].set_index(['model','climate','Mortality','N_obs'])
            res_tables_formatted[f,t] = res_tables_formatted[f,t][['r2','AIC','d_AIC','RMSE','ID']].copy()
            res_tables_formatted[f,t].columns = pd.MultiIndex.from_tuples([(i,tpe) for i in 
                                                                           res_tables_formatted[f,t].columns])
        
        df_mort = pd.concat(res_tables_formatted[f], axis=1)
        df_mort = df_mort.reset_index(drop=False)
        df_mort = df_mort[['Mortality','model','r2','AIC','d_AIC','RMSE','ID','N_obs','climate']].copy()
        df_melt = df_mort.melt(ignore_index=False)
        idx_r2  = df_melt[df_melt.variable_0 == 'r2'].sort_values('value',ascending=False).index.unique()
        df_mort = df_mort.reindex(idx_r2).reset_index(drop=True).copy()
        df_mort.set_index(['Mortality','model'],inplace=True)
        df_res_mort = pd.concat([df_res_mort,df_mort],axis=0)

# Sort by mortality group
df_res_mort = df_res_mort.reset_index(drop=False)
df_res_mort.Mortality = df_res_mort.Mortality.astype("category")
df_res_mort.Mortality = df_res_mort.Mortality.cat.set_categories(mortality_groups)
df_res_mort['Level'] = 'Species' # Add analysis level
       
df_res_mort = df_res_mort.sort_values(["Mortality"]).set_index(['Level','Mortality','model'])
display(df_res_mort)
with open('spe_reg_results/'+date_exp_str+'_'+str(alpha)+'_SPE_best_models_dAIC.pkl','wb') as f:
    pickle.dump(df_res_mort,f)

r2        \
                                                            count cmass   
Level   Mortality model                                                   
Species M_All     MTF ~ Temp + Aridity + Conifer + DBH_mean  0.65   NaN   
                  MTF ~ Temp + Aridity + Conifer + DBH_mean   NaN  0.63   
        M_Fire    MTF ~ DBH_mean                             0.71  0.66   
                  MTF ~ Temp + Aridity + Managed              NaN  0.66   
        M_NoFire  MTF ~ Temp + Conifer + DBH_mean            0.54  0.54   
                  MTF ~ Temp + Conifer                        0.5   NaN   
        M_Other   MTF ~ Temp + Conifer + DBH_mean            0.58  0.58   
                  MTF ~ CDI + Managed                        0.57  0.55   
                  MTF ~ CDI + Conifer                        0.53   NaN   
                  MTF ~ Temp + Conifer                       0.53   NaN   

                                                               AIC         \
                                                             count  cmass   
Level   Mortality model                                                     
Species M_All     MTF ~ Temp + Aridity + Conifer + DBH_mean  190.9    NaN   
                  MTF ~ Temp + Aridity + Conifer + DBH_mean    NaN  214.2   
        M_Fire    MTF ~ DBH_mean                              72.3   82.7   
                  MTF ~ Temp + Aridity + Managed               NaN   83.9   
        M_NoFire  MTF ~ Temp + Conifer + DBH_mean            110.5    123   
                  MTF ~ Temp + Conifer                         116    NaN   
        M_Other   MTF ~ Temp + Conifer + DBH_mean             94.8  105.7   
                  MTF ~ CDI + Managed                         95.9  108.7   
                  MTF ~ CDI + Conifer                        100.6    NaN   
                  MTF ~ Temp + Conifer                       100.6    NaN   

                                                            d_AIC        RMSE  \
                                                            count cmass count   
Level   Mortality model                                                         
Species M_All     MTF ~ Temp + Aridity + Conifer + DBH_mean     0   NaN  25.7   
                  MTF ~ Temp + Aridity + Conifer + DBH_mean   NaN     0   NaN   
        M_Fire    MTF ~ DBH_mean                                0     0  38.3   
                  MTF ~ Temp + Aridity + Managed              NaN   1.2   NaN   
        M_NoFire  MTF ~ Temp + Conifer + DBH_mean               0     0  12.4   
                  MTF ~ Temp + Conifer                        5.5   NaN  12.4   
        M_Other   MTF ~ Temp + Conifer + DBH_mean               0     0  12.6   
                  MTF ~ CDI + Managed                         1.1     3  12.7   
                  MTF ~ CDI + Conifer                         5.8   NaN  13.3   
                  MTF ~ Temp + Conifer                        5.8   NaN  12.5   

                                                                     ID        \
                                                            cmass count cmass   
Level   Mortality model                                                         
Species M_All     MTF ~ Temp + Aridity + Conifer + DBH_mean   NaN     1   NaN   
                  MTF ~ Temp + Aridity + Conifer + DBH_mean  43.7   NaN     1   
        M_Fire    MTF ~ DBH_mean                             67.8     1     0   
                  MTF ~ Temp + Aridity + Managed             78.3   NaN     2   
        M_NoFire  MTF ~ Temp + Conifer + DBH_mean            18.9     3     3   
                  MTF ~ Temp + Conifer                        NaN    11   NaN   
        M_Other   MTF ~ Temp + Conifer + DBH_mean            19.0     2     2   
                  MTF ~ CDI + Managed                        19.5     3     4   
                  MTF ~ CDI + Conifer                         NaN     8   NaN   
                  MTF ~ Temp + Conifer                     